In [4]:
from google.colab import drive
from pathlib import Path
import os

drive.mount("/content/drive")
os.chdir("/content/drive/MyDrive/gb-flex-desk")

Path("data/raw").mkdir(parents=True, exist_ok=True)
Path("data/processed").mkdir(parents=True, exist_ok=True)

print("cwd:", Path.cwd())

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
cwd: /content/drive/MyDrive/gb-flex-desk


In [5]:
import requests
import pandas as pd

BASE = "https://data.elexon.co.uk/bmrs/api/v1"

def system_prices_day(settlement_date: str) -> pd.DataFrame:
    url = f"{BASE}/balancing/settlement/system-prices/{settlement_date}"
    r = requests.get(url, params={"format": "json"}, timeout=30)
    r.raise_for_status()
    return pd.DataFrame(r.json()["data"])

def add_sp_timestamp(df: pd.DataFrame) -> pd.DataFrame:
    d = pd.to_datetime(df["settlementDate"])
    sp = df["settlementPeriod"].astype(int)
    out = df.copy()
    out["ts"] = d + pd.to_timedelta((sp - 1) * 30, unit="min")
    return out.sort_values("ts")

df = add_sp_timestamp(system_prices_day("2024-01-02"))
df.head()

,settlementDate,settlementPeriod,startTime,createdDateTime,systemSellPrice,systemBuyPrice,bsadDefaulted,priceDerivationCode,reserveScarcityPrice,netImbalanceVolume,...,replacementPriceReferenceVolume,totalAcceptedOfferVolume,totalAcceptedBidVolume,totalAdjustmentSellVolume,totalAdjustmentBuyVolume,totalSystemTaggedAcceptedOfferVolume,totalSystemTaggedAcceptedBidVolume,totalSystemTaggedAdjustmentSellVolume,totalSystemTaggedAdjustmentBuyVolume,ts
0,2024-01-02,1,2024-01-02T00:00:00Z,2024-01-02T00:52:15Z,-46.32,-46.32,False,N,NaN,-602.65700,...,NaN,875.0000,-995.4170,-997.24,515.0,875.0000,-995.4170,-996.24,515.0,2024-01-02 00:00:00
1,2024-01-02,2,2024-01-02T00:30:00Z,2024-01-02T01:22:26Z,-44.76,-44.76,False,N,NaN,-552.96280,...,NaN,899.0077,-969.7305,-997.24,515.0,899.0077,-969.7305,-996.24,515.0,2024-01-02 00:30:00
2,2024-01-02,3,2024-01-02T01:00:00Z,2024-01-02T01:52:14Z,-31.00,-31.00,False,N,NaN,-323.01172,...,NaN,1386.4987,-842.2475,-1382.24,515.0,1386.4987,-841.2475,-1382.24,515.0,2024-01-02 01:00:00
3,2024-01-02,4,2024-01-02T01:30:00Z,2024-01-02T02:22:45Z,-44.76,-44.76,False,N,NaN,-466.87740,...,NaN,1238.3628,-838.0308,-1382.24,515.0,1238.3628,-838.0308,-1381.24,515.0,2024-01-02 01:30:00
4,2024-01-02,5,2024-01-02T02:00:00Z,2024-01-02T02:52:24Z,-44.76,-44.76,False,N,NaN,-385.05790,...,NaN,977.9985,-817.3164,-1060.74,515.0,977.9985,-817.3164,-1059.74,515.0,2024-01-02 02:00:00


In [6]:
df.to_parquet("data/raw/system_prices_2024-01-02.parquet", index=False)
print("saved:", Path("data/raw/system_prices_2024-01-02.parquet").resolve())

saved: /content/drive/MyDrive/gb-flex-desk/data/raw/system_prices_2024-01-02.parquet
